## <font color='darkblue'>Preface</font>
* <font size='3ptx'><b><a href='#sect1'>Overloading</a></b></font>
* <font size='3ptx'><b><a href='#sect2'>Single Dispatch</a></b></font>
* <font size='3ptx'><b><a href='#sect3'>Multiple Dispatch</a></b></font>
* <font size='3ptx'><b><a href='#sect4'>Closing Thoughts</a></b></font>
<br/>

([article source](https://martinheinz.dev/blog/50)) <font size='3ptx'><b>Function overloading is a common programming pattern which seems to be reserved to statically-typed, compiled languages</b></font>. Yet there's an easy way to implement it in Python with help of Multiple Dispatch or as it's called in Python multimethods.

<a id='sect1'></a>
## <font color='darkblue'>Overloading</font>
<font size='3ptx'><b>First things first - you might be asking, how can we implement method overloading in Python when we all know that it's not possible?</b></font> Well, even though Python is dynamically-typed language and therefore cannot have proper method overloading as that requires the language to be able to discriminate between types at compile-time, we can still implement it in a bit different way that is suitable for dynamically-typed languages.

<b>This approach is called <font color='darkblue'>Multiple Dispatch</font> or <font color='darkblue'>multimethods</font>, where the interpreter differentiates between multiple implementations of a function/method at runtime based on dynamically determined types.</b> To be more precise, the language uses types of arguments passed to a function during its invocation to dynamically choose which one of the multiple function implementations to use (<font color='brown'>or dispatch</font>).

Now you might be thinking: "Do we really need this though? If it can't be implemented normally, maybe we shouldn't use it in Python..." Yea, valid point, but <b>there are good reasons to want to implement some form of function/method overloading in Python. It's powerful tool that can make code more concise, readable and minimise its complexity</b>. Without multimethods though, the "obvious way" to do this is using type inspection with <a href='https://docs.python.org/3/library/functions.html#isinstance'>isinstance()</a>. This is very ugly, brittle solution that is closed to extension and I would call it an anti-pattern.

Besides that, there already is method overloading in Python for operators and methods like [len()](https://docs.python.org/3/library/functions.html#len) or new() using so-called <b>dunder</b> or <b>magic methods</b> (<font color='brown'>see docs [here](https://docs.python.org/3/reference/datamodel.html#special-method-names)</font>) and we all use that quite often, so why not use proper overloading for all the function, right?

**So, now we know that we can kind-of implement overloading in Python, so how exactly do we do that?**

<a id='sect2'></a>
## <font color='darkblue'>Single Dispatch</font>
<font color='darkred'><b>Above we spoke about Multiple Dispatch, but Python doesn't support this out-of-the box, or in other words Multiple Dispatch is not a feature of Python standard library</b></font>. What is available to us however, is called <b><font color='darkblue'>Single Dispatch</font></b>, so let's begin with this simpler case first.

**The only actual difference between multi and single dispatch is number of arguments which we can overload.** So, for this implementation in standard library it's just one.

The function (<font color='brown'>and decorator</font>) that provides this feature is called [**singledispatch**](https://docs.python.org/3/library/functools.html#functools.singledispatch) and can be found in [**functools**](https://docs.python.org/3/library/functools.html#module-functools) module.

This whole concept is best explained with some examples. There are many "academic" examples of overloading functions (<font color='brown'>geometric shapes, addition, subtraction...</font>) that we've probably all seen already. Rather than going over that, let's see some practical examples. So, here's first example for [singledispatch](https://docs.python.org/3/library/functools.html#functools.singledispatch) to format dates, times and datetimes:

In [1]:
from functools import singledispatch, singledispatchmethod
from datetime import date, datetime, time

In [2]:
@singledispatch
def format(arg):
    return arg

@format.register
def _(arg: date):
    return f"{arg.day}-{arg.month}-{arg.year}"

@format.register
def _(arg: datetime):
    return f"{arg.day}-{arg.month}-{arg.year} {arg.hour}:{arg.minute}:{arg.second}"

@format.register(time)
def _(arg):
    return f"{arg.hour}:{arg.minute}:{arg.second}"

In [3]:
# today
print(format("today"))

# 26-5-2021
print(format(date(2021, 5, 26)))

# 26-5-2021 17:25:10
print(format(datetime(2021, 5, 26, 17, 25, 10)))

# 19:22:15
print(format(time(19, 22, 15)))

today
26-5-2021
26-5-2021 17:25:10
19:22:15


We begin by defining the base `format` function that is going to be overloaded. This function is decorated with [**@singledispatch**](https://docs.python.org/3/library/functools.html#functools.singledispatch) and provides base implementation, which is used if no better options is available. Next, we define individual functions for each type that we want to overload - in this case [**date**](https://docs.python.org/3/library/datetime.html#date-objects), [**datetime**](https://docs.python.org/3/library/datetime.html#datetime-objects) and [**time**](https://docs.python.org/3/library/datetime.html#time-objects) - each of these have name `_` (<font color='brown'>underscore</font>) because they will be called (<font color='brown'>dispatched</font>) through the `format` method anyway, so no need to give them useful names. Each of them is also decorated with <b><font color='orange'>@format.register</font></b> which attaches them to the previously mentioned format function. Then, to make it possible to differentiate between types, we have two options - we can use type annotations - as demonstrated in first two cases or explicitly add the type to decorator as with the last one from the example.

**In some cases it might make sense to use same implementation for multiple types** - for example for number types such as `int` and `float` - for these situations decorator stacking is allowed, meaning that you can list (<font color='brown'>stack</font>) multiple <font color='orange'><b>@format.register(type)</b></font> lines to associate a function with all the valid types.

Besides ability to overload basic functions, [**functools**](https://docs.python.org/3/library/functools.html#module-functools) module contains also [**singledispatchmethod**](https://docs.python.org/3/library/functools.html#functools.singledispatchmethod) that can be applied to methods of a class. Example of that could be the following:

In [2]:
class Formatter:
    @singledispatchmethod
    def format(self, arg):
        raise NotImplementedError(f"Cannot format value of type {type(arg)}")

    @format.register
    def _(self, arg: date):
        return f"{arg.day}-{arg.month}-{arg.year}"

    @format.register
    def _(self, arg: time):
        return f"{arg.hour}:{arg.minute}:{arg.second}"

In [3]:
f = Formatter()

# 26-5-2021
print(f.format(date(2021, 5, 26)))

# 19:22:15
print(f.format(time(19, 22, 15)))

26-5-2021
19:22:15


<a id='sect3'></a>
## <font color='darkblue'>Multiple Dispatch</font>
Oftentimes <b><font color='darkblue'>Single Dispatch</font></b> won't be sufficient and you might need the proper Multiple Dispatch functionality. This is available from [**multipledispatch**](https://pypi.org/project/multipledispatch/) module which can be found here and can be installed with <font color='blue'>pip install multipledispatch</font>.

In [4]:
!pip install multipledispatch

This module and it's decorator - <b><font color='orange'>@dispatch</font></b>, behaves very similarly to the [**@singledispatch**](https://docs.python.org/3/library/functools.html#functools.singledispatch) in the standard library. Only actual difference is that it can take multiple types as arguments:

In [5]:
from multipledispatch import dispatch

In [6]:
@dispatch(list, str)
def concatenate(a, b):
    a.append(b)
    return a

@dispatch(str, str)
def concatenate(a, b):
    return a + b

@dispatch(str, int)
def concatenate(a, b):
    return a + str(b)

In [7]:
# ['a', 'b', 'c']
print(concatenate(["a", "b"], "c"))

# HelloWorld
print(concatenate("Hello", "World"))

# a1
print(concatenate("a", 1))

['a', 'b', 'c']
HelloWorld
a1


The above snippet shows how we can use <b><font color='orange'>@dispatch</font></b> decorator to overload multiple arguments, for example to implement `concatenation` of various types. As you probably noticed, with [**multipledispatch**](https://pypi.org/project/multipledispatch/) library we didn't need to define and register base function, rather we created multiple functions with same name. **If we wanted to provide base implementation, we could use <font color='orange'>@dispatch(object, object)</font> which would catch any non-specific argument types**.

The previous examples shows proof-of-concept, but if we wanted to really implement such `concatenate` function, we would need to make it much more generic. This can be solved with use of [**union**](https://docs.python.org/3/library/typing.html#typing.Union) types. In this specific example we could change the first function as follows:

In [10]:
@dispatch((list, tuple), (str, int))
def concatenate(a, b):
    print(f'concatenate a={a} with b={b}')
    return list(a) + [b]

In [11]:
# # ['a', 'b', 'c']
print(concatenate(["a", "b"], "c"))

# ['a', 'b', 1]
print(concatenate(("a", "b"), 1))

concatenate a=['a', 'b'] with b=c
['a', 'b', 'c']
concatenate a=('a', 'b') with b=1
['a', 'b', 1]


This would make it so that first argument of the function could be any of `list` or `tuple`, while second one would be `str` or `int`. This is already much better than the previous solution, but it can be further improved using abstract types. Instead of listing all the possible sequences, we can use [**Sequence**](https://docs.python.org/3/library/typing.html#typing.Sequence) abstract type (<font color='brown'>assuming that our implementation can handle it</font>) which covers things like `list`, `tuple` or `range`:

In [12]:
from collections.abc import Sequence

In [13]:
@dispatch(Sequence, (str, int))
def concatenate(a, b):
    print(f"Concatenate sequence a={a} with b={b}")
    return list(a) + [b]

In [14]:
print(concatenate(range(0, 10), 1))

Concatenate sequence a=range(0, 10) with b=1
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1]


If you want to take this approach, then it's good to take a look at [**collections.abc**](https://docs.python.org/3/library/collections.abc.html) module and see which container data-type best suits your needs. Mostly to make sure that your function will be able to handle all the types that fall into the chosen container.

<b><font color='darkred'>All this mixing and matching of argument types is convenient, but can also cause ambiguities when choosing suitable function for some specific set of parameters</font></b>. Fortunately, [**multipledispatch**](https://pypi.org/project/multipledispatch/) provides <font color='blue'><b>AmbiguityWarning</b></font> which is raised if ambiguous behaviour is possible:

In [54]:
@dispatch(str, str)
def some_func(a, b):
    return 'p1'

@dispatch(str, range)
def some_func(a, b):
    return 'p2'

@dispatch(str, object)
def some_func(a, b):
    return 'p3'

In [56]:
assert some_func('a', 'b') == 'p1'
assert some_func('a', range(1,2)) == 'p2'
assert some_func('a', lambda a: a) == 'p3'

<a id='sect4'></a>
## <font color='darkblue'>Closing Thoughts</font>
In this article we went over a simple, yet powerful concept which I rarely see being used in Python, which is a shame considering that it can greatly improve code readability and get rid of anti-patters like type inspection using [isinstance()](https://docs.python.org/3/library/functions.html#isinstance). Also, I hope you would agree that this approach to function overloading should be considered the "obvious way" and I hope that you will make use of it when needed.

If you want to dive deeper into this topic and get your hands dirty you can implement multimethods yourselves as shown in [Guido's article](https://www.artima.com/weblogs/viewpost.jsp?thread=101605) - this can be a good exercise to understand how multiple dispatch actually works.

Finally, I should also probably mention that this article omits examples of the well-known [operator overloading](https://docs.python.org/3/reference/datamodel.html#basic-customization) which I mentioned in the beginning as well as some approaches for overloading constructors for example using [factories](https://stackoverflow.com/a/141777). So, in case that's what you're looking for, go check out these links/resources, which give good overview on there topics.

## <font color='darkblue'>Supplement</font>
* [RealPython - Providing Multiple Constructors With @singledispatchmethod](https://realpython.com/python-multiple-constructors/#providing-multiple-constructors-with-singledispatchmethod)